# ETL Pipeline for Immigration Data

This notebook demonstrates an ETL (Extract, Transform, Load) pipeline for immigration data. The pipeline includes:

1. Adding new records to the full dataset
2. Transforming the full dataset (enrichment, structural changes, categorization)
3. Extracting and transforming only the latest (incremental) record
4. Saving and displaying results as tables


In [ ]:
import pandas as pd
from tabulate import tabulate

# Load the dataset
file_path ="K:\Code Projects\Cloned Projects from jpchawanda1 Github\ETL_Extract_Justice_Chawanda_670444\Immigration_Data.csv"
data = pd.read_csv(file_path)

# Display basic stats
print(f"Rows: {data.shape[0]}, Columns: {data.shape[1]}")
print(tabulate(data, headers='keys', tablefmt='grid'))

# Print extraction message
print(f"Extracted {data.shape[0]} rows fully.")

In [ ]:
import os
from datetime import datetime
import pandas as pd

# Simulate a last extraction time
last_extraction_file = "last_extraction.txt"
if not os.path.exists(last_extraction_file):
    with open(last_extraction_file, "w") as f:
        f.write("2025-06-01 00:00:00")  # Initial extraction time

# Read the last extraction time
with open(last_extraction_file, "r", encoding="utf-8") as f:
    content = f.read().strip()
    try:
        last_extraction_time = datetime.strptime(content, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        # If the file content is invalid, reset to a default time
        last_extraction_time = datetime(2025, 6, 1, 0, 0, 0)
        with open(last_extraction_file, "w", encoding="utf-8") as fw:
            fw.write(last_extraction_time.strftime("%Y-%m-%d %H:%M:%S"))

# Filter new or updated records using mixed datetime formats
new_data = data[pd.to_datetime(data['timestamp'], format='mixed') > last_extraction_time]

# Print the number of rows extracted incrementally
print(f"Extracted {new_data.shape[0]} rows incrementally since last check.")

# Update the last extraction time
with open(last_extraction_file, "w") as f:
    f.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# Display the last recorded timestamp
if not data.empty:
    last_timestamp = data['timestamp'].iloc[-1]
    description = f"The last record was added on: {last_timestamp}"
    # Write the description and last timestamp to the text file
    with open(last_extraction_file, "w") as f:
        f.write(description)
else:
    description = "The dataset is empty."
    with open(last_extraction_file, "w") as f:
        f.write(description)

In [ ]:
# Ask the user if they want to add a new record
add_record = input("Do you want to add a new record? (yes/no): ").strip().lower()
if add_record == 'yes':
    # Collect new record details from the user
    new_record = {
        "immigrant_id": input("Enter immigrant ID: "),
        "passport_number": input("Enter passport number: "),
        "name": input("Enter name: "),
        "country": input("Enter country: "),
        "purpose_of_visit": input("Enter purpose of visit: "),
        "contact": input("Enter contact: "),
        "payment_status": input("Enter payment status: "),
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    # Append the new record to the dataset
    data = pd.concat([data, pd.DataFrame([new_record])], ignore_index=True)

    # Save the updated dataset back to the file
    data.to_csv(file_path, index=False)

    # Update the last_extraction.txt file with the description and timestamp of the very last record
    last_timestamp = data['timestamp'].iloc[-1]
    description = f"The last record was added on: {last_timestamp}"
    with open(last_extraction_file, "w") as f:
        f.write(description)

    print("New record added successfully and timestamp updated!")
else:
    print("No new record added.")

## Section 4: Transform Full Data

In this section, we will apply three transformations to the full dataset:
1. **Enrichment**: Add a calculated 'age' column from the 'date_of_birth' column.
2. **Structural**: Standardize the 'timestamp' column to always include seconds.
3. **Categorization**: Bin records based on the 'country' column.

The transformed data will be saved as `transformed_full.csv`.

---


In [ ]:
import pandas as pd
import numpy as np

# Load the full dataset (replace with your actual file path if needed)
full_data = pd.read_csv("K:\Code Projects\Cloned Projects from jpchawanda1 Github\ETL_Extract_Justice_Chawanda_670444\Immigration_Data.csv")  # Change 'full.csv' to your actual full dataset filename

# Add a 'date_of_birth' column with random dates between 1970-01-01 and 2005-12-31 if not present
if 'date_of_birth' not in full_data.columns:
    np.random.seed(42)
    start_date = pd.to_datetime('1970-01-01')
    end_date = pd.to_datetime('2005-12-31')
    num_rows = len(full_data)
    random_days = np.random.randint(0, (end_date - start_date).days, num_rows)
    full_data['date_of_birth'] = start_date + pd.to_timedelta(random_days, unit='D')  # keep as datetime64[ns]

# Calculate 'age' from 'date_of_birth'
today = pd.to_datetime('today')
full_data['age'] = (today - pd.to_datetime(full_data['date_of_birth'])).dt.days // 365

# Standardize 'timestamp' column to include seconds
if 'timestamp' in full_data.columns:
    full_data['timestamp'] = pd.to_datetime(full_data['timestamp'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# Bin countries into continent-based regions
def country_to_region(country):
    africa = ['Nigeria', 'Kenya', 'South Africa', 'Egypt', 'Ghana']
    europe = ['United Kingdom', 'France', 'Germany', 'Italy', 'Spain']
    asia = ['China', 'India', 'Japan', 'Pakistan', 'Bangladesh']
    north_america = ['United States', 'Canada', 'Mexico']
    south_america = ['Brazil', 'Argentina', 'Colombia']
    oceania = ['Australia', 'New Zealand']
    if country in africa:
        return 'Africa'
    elif country in europe:
        return 'Europe'
    elif country in asia:
        return 'Asia'
    elif country in north_america:
        return 'North America'
    elif country in south_america:
        return 'South America'
    elif country in oceania:
        return 'Oceania'
    else:
        return 'Other'

if 'country' in full_data.columns:
    full_data['country_group'] = full_data['country'].apply(country_to_region)

# Save the transformed full dataset
full_data.to_csv('transformed_full.csv', index=False)

print('Added Date of Birth, calculated age, standardized timestamp, and grouped countries into regions for full transformation.')


## Section 5: Transform Incremental Data

We will apply the same transformations to the incremental (new or updated) data and save the result as `transformed_incremental.csv`.

In [ ]:
# Load the full dataset (replace with your actual file path if needed)
# Extract only the last (newest) record for incremental processing
incremental_data = full_data.tail(1).copy()

# 1. Enrichment: Add calculated 'age' from 'date_of_birth' column
if 'date_of_birth' in incremental_data.columns:
    today = pd.to_datetime('today')
    incremental_data['date_of_birth'] = pd.to_datetime(incremental_data['date_of_birth'], errors='coerce')
    incremental_data['age'] = (today - incremental_data['date_of_birth']).dt.days // 365

# 2. Structural: Standardize 'timestamp' column to include seconds
if 'timestamp' in incremental_data.columns:
    incremental_data['timestamp'] = pd.to_datetime(incremental_data['timestamp'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

# 3. Categorization: Bin records based on 'country'
if 'country' in incremental_data.columns:
    # Ensure access to the global country_bins variable
    incremental_data['country_group'] = incremental_data['country'].map(globals().get('country_bins', {})).fillna('Other')

# Save the transformed incremental dataset
incremental_data.to_csv('transformed_incremental.csv', index=False)

print('Transformed incremental dataset saved as transformed_incremental.csv')

In [ ]:
import numpy as np
import pandas as pd

# Add a 'date_of_birth' column with random dates between 1970-01-01 and 2005-12-31
np.random.seed(42)
start_date = pd.to_datetime('1970-01-01')
end_date = pd.to_datetime('2005-12-31')
num_rows = len(incremental_data)
random_days = np.random.randint(0, (end_date - start_date).days, num_rows)
incremental_data['date_of_birth'] = start_date + pd.to_timedelta(random_days, unit='D')  # keep as datetime64[ns]

# Calculate 'age' from 'date_of_birth'
today = pd.to_datetime('today')
incremental_data['age'] = ((today - incremental_data['date_of_birth']).dt.days // 365).astype(int)

# Bin countries into continent-based regions
def country_to_region(country):
    africa = ['Nigeria', 'Kenya', 'South Africa', 'Egypt', 'Ghana', 'Malawi', 'Zimbabwe']
    europe = ['United Kingdom', 'France', 'Germany', 'Italy', 'Spain']
    asia = ['China', 'India', 'Japan', 'Pakistan', 'Bangladesh']
    north_america = ['United States', 'Canada', 'Mexico']
    south_america = ['Brazil', 'Argentina', 'Colombia']
    oceania = ['Australia', 'New Zealand']
    if country in africa:
        return 'Africa'
    elif country in europe:
        return 'Europe'
    elif country in asia:
        return 'Asia'
    elif country in north_america:
        return 'North America'
    elif country in south_america:
        return 'South America'
    elif country in oceania:
        return 'Oceania'
    else:
        return 'Other'

if 'country' in incremental_data.columns:
    incremental_data['country_group'] = incremental_data['country'].apply(country_to_region)

print('Added Date of Birth, calculated age, standardized timestamp, and grouped countries into regions for incremental transformation')

### Saving and Displaying Transformed Data

This cell performs the following actions:

- Saves the transformed `full_data` and `incremental_data` DataFrames to CSV files named `transformed_full.csv` and `transformed_incremental.csv`.
- Prints confirmation messages indicating that the files have been saved.
- Displays the first 10 rows of both the full and incremental datasets as formatted tables using the `tabulate` library for easy viewing.

This step ensures that the results of the ETL process are both persisted and visually inspected for correctness.

In [68]:
# 4. Save and display the results as tables

from tabulate import tabulate
import numpy as np
from datetime import datetime

# Define the desired column order
column_order = ['immigrant_id','passport_number','name','date_of_birth','age','contact','country',
                'country_group','purpose_of_visit','payment_status','timestamp']

# Reorder columns for full_data and incremental_data if all columns exist
def reorder_columns(df, order):
    cols = [col for col in order if col in df.columns] + [col for col in df.columns if col not in order]
    return df[cols]

# Ensure timestamp is present and in the correct format for all records in full_data
if 'timestamp' not in full_data.columns:
    full_data['timestamp'] = np.nan
full_data['timestamp'] = full_data['timestamp'].fillna(datetime.now().strftime('%d-%m-%Y %H:%M:%S'))
full_data['timestamp'] = pd.to_datetime(full_data['timestamp'], errors='coerce').dt.strftime('%d-%m-%Y %H:%M:%S')

full_data = reorder_columns(full_data, column_order)

full_data_fixed = (full_data.tail(10))

# Save the updated DataFrames to CSV files
full_data.to_csv('transformed_full.csv', index=False)

print('Saved full_data to transformed_full.csv.')

print('\nFull Data (first 10 rows):')
print(tabulate(full_data_fixed, headers='keys', tablefmt='github', showindex=False))

Saved full_data to transformed_full.csv.

Full Data (first 10 rows):
| immigrant_id   | passport_number   | name             | date_of_birth       |   age | contact                 | country       | country_group   | purpose_of_visit   | payment_status   | timestamp           |
|----------------|-------------------|------------------|---------------------|-------|-------------------------|---------------|-----------------|--------------------|------------------|---------------------|
| IM0094         | PPP45678910       | Anna Weber       | 1990-11-21 00:00:00 |    34 | anna.weber@email.de     | Germany       | Europe          | Tourism            | Paid             | 09-06-2025 18:52:00 |
| IM0095         | QQQ56789021       | James O'Connor   | 2002-10-30 00:00:00 |    22 | james.oconnor@email.ie  | Ireland       | Other           | Study              | Paid             | 09-06-2025 21:08:00 |
| IM0096         | RRR67890132       | Yusra Begum      | 1995-12-03 00:00:00 |    29 | yus

In [ ]:
# 4. Save and display the results as tables

from tabulate import tabulate
import numpy as np
from datetime import datetime

# Define the desired column order
column_order = ['immigrant_id','passport_number','name','date_of_birth','age','contact','country',
                'country_group','purpose_of_visit','payment_status','timestamp']

# Reorder columns for full_data and incremental_data if all columns exist
def reorder_columns(df, order):
    cols = [col for col in order if col in df.columns] + [col for col in df.columns if col not in order]
    return df[cols]

incremental_data = reorder_columns(incremental_data, column_order)

print('Saved incremental_data to transformed_incremental.csv')

print('\nIncremental Data (latest row):')
incremental_data = (incremental_data.head(10))
print(tabulate(incremental_data, headers='keys', tablefmt='github', showindex=False))